**Morro Bay Triathlon Data Collection**

Via webscraping off of https://www.morrobaytri.com/

The only readily available data for webscraping was from 2015-2017. There is no information about the course in those years but it must be similar enough to recent years. 

In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
import math

In [5]:
# Webscrape the data from 2017 Morro Bay Triathlon

response = requests.get("https://www.synergyracetiming.com/2017-morro-bay-triathlon/")

In [6]:
soup = BeautifulSoup(response.text, "html.parser")

table = soup.find('table')

In [7]:
rows = table.find_all('tr')

data = []

for row in rows:
    columns = row.find_all(['th', 'td'])
    row_data = [column.text for column in columns]
    data.append(row_data)

df_2017 = pd.DataFrame(data[1:], columns=data[0])

In [9]:
# Let's repeat this process for the next couple of years

response = requests.get("https://www.synergyracetiming.com/2016-morro-bay-triathlon/")

In [10]:
soup = BeautifulSoup(response.text, "html.parser")

table = soup.find('table')

rows = table.find_all('tr')

data = []

for row in rows:
    columns = row.find_all(['th', 'td'])
    row_data = [column.text for column in columns]
    data.append(row_data)

df_2016 = pd.DataFrame(data[1:], columns=data[0])

In [11]:
# Now for 2015

response = requests.get("https://www.synergyracetiming.com/morro-bay-triathlon/")

In [12]:
soup = BeautifulSoup(response.text, "html.parser")

table = soup.find('table')

rows = table.find_all('tr')

data = []

for row in rows:
    columns = row.find_all(['th', 'td'])
    row_data = [column.text for column in columns]
    data.append(row_data)

df_2015 = pd.DataFrame(data[1:], columns=data[0])

Now that I've gotten all the data frames from 2015, 2016, and 2017 webscraped. It's time to clean them. Mainly, I want to turn some variables into numeric and I need to fix some weird errors with others.

I also want to narrow all these data frames to points of Males between the age of 20-29.

In [13]:
df_2017["Age"] = df_2017["Age"].astype(int)

df_2017 = df_2017[df_2017["Gender"] == "M"]

columns_to_convert = ["Gender Place", "Overall Place", "Age Group Place", "Swim Rank", "T1 Rank", "Bike Rank", "T2 Rank", "Run Rank"]

df_2017[columns_to_convert] = df_2017[columns_to_convert].replace("INC", np.nan)

df_2017[columns_to_convert] = df_2017[columns_to_convert].apply(pd.to_numeric, errors='coerce')

df_2017["Swim Time"] = df_2017["Swim Time"].replace("", np.nan)

In [14]:
def convert_time_to_minutes(time_str):
    if isinstance(time_str, str):
        hours, minutes, seconds = map(int, time_str.split(":"))
        total_minutes = (hours * 60) + minutes + math.ceil(seconds / 60)
        return total_minutes
    else:
        return time_str

In [15]:
def convert_mmss_to_minutes(time_str):

    if time_str.lower() != 'nan' and len(time_str) <= 5:
        minutes, seconds = map(int, time_str.split(":"))
        total_minutes = minutes + math.ceil(seconds / 60)
        return total_minutes
    else:
        return time_str

In [16]:
df_2017["Swim Time"] = df_2017["Swim Time"].astype(str)

column_name = "Swim Time"
for i in range(len(df_2017[column_name])):
    if len(df_2017[column_name].iloc[i]) > 5:
        df_2017[column_name].iloc[i] = df_2017[column_name].iloc[i][:-3]

<positron-console-cell-16>:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<positron-console-cell-16>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:/

In [17]:
df_2017.replace('', pd.NA, inplace=True)
df_2017 = df_2017.dropna()

In [18]:
df_2017["Time"] = df_2017["Time"].apply(convert_time_to_minutes)
df_2017["Swim Time"] = df_2017["Swim Time"].apply(convert_mmss_to_minutes)

In [19]:
df_2017["T1 Time"] = df_2017["T1 Time"].astype(str)

column_name = "T1 Time"
for i in range(len(df_2017[column_name])):
    if len(df_2017[column_name].iloc[i]) > 5:
        df_2017[column_name].iloc[i] = df_2017[column_name].iloc[i][:-3]


df_2017["T1 Time"] = df_2017["T1 Time"].apply(convert_mmss_to_minutes)

df_2017.loc[df_2017["T1 Time"] > 20, "T1 Time"] = pd.NaT

<positron-console-cell-19>:11: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'NaT' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.


In [20]:
df_2017["Bike Time"] = df_2017["Bike Time"].astype(str)

column_name = "Bike Time"
for i in range(len(df_2017[column_name])):
    if len(df_2017[column_name].iloc[i]) == 8:
        df_2017[column_name].iloc[i] = df_2017[column_name].iloc[i][:-3]

df_2017["Bike Time"] = df_2017["Bike Time"].apply(convert_mmss_to_minutes)

df_2017["Bike Time"] = df_2017["Bike Time"].apply(convert_time_to_minutes)

<positron-console-cell-20>:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<positron-console-cell-20>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:/

<positron-console-cell-20>:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<positron-console-cell-20>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:/

<positron-console-cell-20>:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<positron-console-cell-20>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:/

In [21]:
df_2017["T2 Time"] = df_2017["T2 Time"].apply(convert_mmss_to_minutes)

In [22]:
df_2017["Run Time"] = df_2017["Run Time"].astype(str)

column_name = "Run Time"
for i in range(len(df_2017[column_name])):
    if len(df_2017[column_name].iloc[i]) == 8:
        df_2017[column_name].iloc[i] = df_2017[column_name].iloc[i][:-3]

df_2017["Run Time"] = df_2017["Run Time"].apply(convert_mmss_to_minutes)

<positron-console-cell-22>:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<positron-console-cell-22>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https:/

<positron-console-cell-22>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<positron-console-cell-22>:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://

<positron-console-cell-22>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
<positron-console-cell-22>:6: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://

In [23]:
df_2017["Run Time"] = df_2017["Run Time"].apply(convert_time_to_minutes)

In [32]:
roee_morag = pd.DataFrame({
    "Overall Place": ["NA"],
    "First Name": ["Roee"],
    "Last Name": ["Morag"],
    "Bib": [715],
    "Time": [122],
    "Gender": ["M"],
    "Gender Place": [116],
    "Age": [21],
    "Age Group Place": [30],
    "Swim Rank": ["NA"],
    "Swim Time": [17],
    "T1 Rank": ["NA"],
    "T1 Time": [8],
    "Bike Rank": ["NA"],
    "Bike Time": [55],
    "T2 Rank": ["NA"],
    "T2 Time": [3],
    "Run Rank": ["NA"],
    "Run Time": ["39"]
})

In [33]:
df_tri = pd.concat([df_2017, roee_morag], ignore_index=True)

In [36]:
df_2016["Age"] = df_2016["Age"].astype(int)
df_2016 = df_2016[df_2016["Gender"] == "M"]

In [37]:
df_2016 = df_2016[['Overall Place', 'First Name', 'Last Name', 'Bib', 'ChipTime', 'Gender',
       'Age', 'Class Pos', 'Swim Time', 'T1 Time', 'Bike Time', 'T2 Time', 'Finish Time']]

# Run time is labeled finish time, weird data entry issue

In [38]:
df_2016.replace('', pd.NA, inplace=True)
df_2016 = df_2016.dropna()

In [39]:
df_2016.at[206, "Swim Time"] = df_2016.at[206, "Swim Time"][:-3]
df_2016.at[208, "Swim Time"] = df_2016.at[208, "Swim Time"][:-3]

In [40]:
df_2016[df_2016["Swim Time"] == "28:08:00"].index
df_2016.at[193, "Swim Time"] = df_2016.at[193, "Swim Time"][:-3]

In [41]:
df_2016["ChipTime"] = df_2016["ChipTime"].apply(convert_time_to_minutes)
df_2016["Swim Time"] = df_2016["Swim Time"].apply(convert_mmss_to_minutes)

In [42]:
df_2016["T1 Time"] = df_2016["T1 Time"].apply(convert_mmss_to_minutes)
df_2016["T2 Time"] = df_2016["T2 Time"].apply(convert_mmss_to_minutes)

In [44]:
df_2016["Bike Time"] = df_2016["Bike Time"].str[:-3]
df_2016["Run Time"] = df_2016["Finish Time"].str[:-3]

In [45]:
df_2016 = df_2016[['Overall Place', 'First Name', 'Last Name', 'Bib', 'ChipTime', 'Gender',
       'Age', 'Class Pos', 'Swim Time', 'T1 Time', 'Bike Time', 'T2 Time', 'Run Time']]

In [46]:
df_2016["Bike Time"] = df_2016["Bike Time"].apply(convert_mmss_to_minutes)
df_2016["Run Time"] = df_2016["Run Time"].apply(convert_mmss_to_minutes)

In [47]:
df_2016.rename(columns={'ChipTime': 'Time'}, inplace=True)
df_2016.rename(columns={'Class Pos': 'Age Group Place'}, inplace=True)

In [48]:
df_tri = df_tri[['Overall Place', 'First Name', 'Last Name', 'Bib', 'Time', 'Gender',
       'Age', 'Age Group Place', 'Swim Time', 'T1 Time', 'Bike Time',
       'T2 Time', 'Run Time']]

In [49]:
df_tri = pd.concat([df_2016, df_tri], ignore_index=True)

In [50]:
df_2015["Age"] = df_2015["Age"].astype(int)
df_2015 = df_2015[df_2015["Gender"] == "M"]

In [51]:
df_2015.rename(columns={'Class Pos': 'Age Group Place'}, inplace=True)
df_2015 = df_2015[['Overall Place', 'First Name', 'Last Name', 'Time', 'Bib', 'Gender',
                   'Age', 'Age Group Place', 'Swim Time', 'Transition Time', 'Bike Time', 'Run Time']]

In [52]:
df_2015.columns = ['Overall Place', 'First Name', 'Last Name', 'Time', 'Bib', 'Gender', 'Age', 'Age Group Place', 'Swim Time',
                   'T1 Time', 'T2 Time', 'Bike Time', 'Run Time']

In [53]:
new_order = ['Overall Place', 'First Name', 'Last Name', 'Bib', 'Time', 'Gender',
             'Age', 'Age Group Place', 'Swim Time', 'T1 Time', 'Bike Time',
              'T2 Time', 'Run Time']

df_2015 = df_2015[new_order]

In [54]:
df_2015.replace('', pd.NA, inplace=True)
df_2015 = df_2015.dropna()

In [55]:
string_indices = df_2015.index[df_2015["Swim Time"].map(type) == str]
string_indices = [54, 78, 100, 103, 106, 116, 124, 129, 130, 133, 134]

In [56]:
df_2015.loc[string_indices, "Swim Time"] = df_2015.loc[string_indices, "Swim Time"].str[:-3]

In [57]:
df_2015["Time"] = df_2015["Time"].apply(convert_time_to_minutes)
df_2015["Swim Time"] = df_2015["Swim Time"].apply(convert_mmss_to_minutes)

In [58]:
df_2015["T1 Time"] = df_2015["T1 Time"].apply(convert_mmss_to_minutes)
df_2015["T2 Time"] = df_2015["T2 Time"].apply(convert_mmss_to_minutes)

In [59]:
df_2015["Bike Time"] = df_2015["Bike Time"].str[:-3]

mask = df_2015["Run Time"].str.len() == 8
df_2015.loc[mask, "Run Time"] = df_2015.loc[mask, "Run Time"].str[:-3]

In [60]:
df_2015["Bike Time"] = df_2015["Bike Time"].apply(convert_mmss_to_minutes)
df_2015["Run Time"] = df_2015["Run Time"].apply(convert_mmss_to_minutes)

In [61]:
df_tri = pd.concat([df_2015, df_tri], ignore_index=True)

In [62]:
df_tri["AgeCat"] = pd.cut(
    df_tri["Age"],
    bins=[10, 20, 30, 40, 50, 60, 70, 80, 120],
    labels=["10-19", "20-29", "30-39", "40-49", "50-59", "60-69", "70-79", "80+"],
    right=False)

In [63]:
df_tri["AgeCat"].value_counts()

AgeCat
30-39    67
20-29    64
40-49    53
50-59    43
60-69    29
10-19    12
70-79     5
80+       0
Name: count, dtype: int64

In [64]:
df_tri["Run Time"] = df_tri["Run Time"].astype(int)

df_tri["T1 Time"] = df_tri["T1 Time"].astype(int)

In [65]:
df_tri = df_tri[df_tri["Run Time"] > 2]
df_tri = df_tri[df_tri["Bike Time"] > 2]

In [66]:
df_tri.to_csv("morro_triathlon.csv")